# Bias Test

In [4]:
import torch

PATH="/export/data4/vzhekova/biases-data/Test_De/Bias"
FASTBPE="/home/vzhekova/fastBPE/fast" # path to the fastBPE tool

## 1) Translation English-German

Sentence: 
- The doctor asked the patient.

In [3]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De


In [21]:
# Dividing tokenized text into subword units

!$FASTBPE applybpe bpe.test.en-de.en test.en-de.en bpecodes.en
!$FASTBPE applybpe bpe.test.en-de.de test.en-de.de bpecodes.en

print('Finished subword.')

Loading codes from bpecodes.en ...
Read 30000 codes from the codes file.
Loading vocabulary from test.en-de.en ...
Read 5 words (5 unique) from text file.
Applying BPE to test.en-de.en ...
Modified 5 words from text file.
Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from test.en-de.de ...
Read 5 words (5 unique) from text file.
Applying BPE to test.en-de.de ...
Modified 5 words from text file.
Finished subword.


In [22]:
# !!! Copy the dict.de to data-bin for translation
!fairseq-preprocess \
    --source-lang en \
    --target-lang de \
    --testpref bpe.test.en-de \
    --srcdict /export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble/dict.en.txt \
    --tgtdict /export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble/dict.de.txt \
    --destdir data-bin_en-de \
    --workers 8

2023-03-16 23:28:06 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin_en-de', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='en', srcdict='/ex

In [23]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De


- Beam search

In [24]:
MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"
NBEST = 10
BEAM = 10

# Generate N hypothesis
!fairseq-generate data-bin_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 256 \
    --memory-efficient-fp16 \
    --remove-bpe > en-de.decode_Beam_10.log

2023-03-16 23:29:40 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

## Backtranslation German-English

- Extract hypothesis

In [25]:
!grep ^H en-de.decode_Beam_10.log | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp.txt

In [26]:
!head ./hyp.txt

Der Arzt fragte den Patienten
Der Arzt fragte die Patientin
Der Arzt befragte den Patienten
Der Arzt bat den Patienten
Die Ärztin fragte den Patienten
Der Arzt hat den Patienten gefragt
Der Arzt fragte den Patienten ,
Der Arzt fragt den Patienten
Der Arzt fragte den Patienten .
Arzt fragte den Patienten


In [27]:
# Dividing tokenized text into subword units

!$FASTBPE applybpe bpe.hyp.de hyp.txt bpecodes.de

print('Finished subword.')

Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from hyp.txt ...
Read 52 words (16 unique) from text file.
Applying BPE to hyp.txt ...
Modified 52 words from text file.
Finished subword.


In [28]:
# !!! Copy the dict.de to data-bin for translation
!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_de-en \
    --workers 8

2023-03-16 23:36:26 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin_de-en', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='de', srcdict='/exp

- Translate hypothesis back to En

In [29]:
MODELS="/export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble"
NBEST = 10
BEAM = 10

# Generate N hypothesis
!fairseq-generate data-bin_de-en  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 256 \
    --memory-efficient-fp16 \
    --remove-bpe > de-en.decode_Beam_10_backtranslation.log

2023-03-16 23:39:18 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

# Ambiguous words Test

In [27]:
PATH="/export/data4/vzhekova/biases-data/Test_De/Ambiguity"

## 1) Translation English-German

Sentence: 
- He drove the car.

In [2]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De/Ambiguous


In [33]:
# Dividing tokenized text into subword units

!$FASTBPE applybpe bpe.test.en-de.en test.en-de.en bpecodes.en
!$FASTBPE applybpe bpe.test.en-de.de test.en-de.de bpecodes.en

print('Finished subword.')

Loading codes from bpecodes.en ...
Read 30000 codes from the codes file.
Loading vocabulary from test.en-de.en ...
Read 4 words (4 unique) from text file.
Applying BPE to test.en-de.en ...
Modified 4 words from text file.
Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from test.en-de.de ...
Read 4 words (4 unique) from text file.
Applying BPE to test.en-de.de ...
Modified 4 words from text file.
Finished subword.


In [34]:
# !!! Copy the dict.de to data-bin for translation
!fairseq-preprocess \
    --source-lang en \
    --target-lang de \
    --testpref bpe.test.en-de \
    --srcdict /export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble/dict.en.txt \
    --tgtdict /export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble/dict.de.txt \
    --destdir data-bin_en-de \
    --workers 8

2023-03-20 17:23:56 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin_en-de', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='en', srcdict='/ex

In [28]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De/Ambiguity


- Beam search

In [29]:
MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"
NBEST = 10
BEAM = 10

# Generate N hypothesis
!fairseq-generate data-bin_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 256 \
    --memory-efficient-fp16 \
    --print-alignment \
    --remove-bpe > en-de.decode_Beam_10.log

2023-03-23 19:03:51 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

## Backtranslation German-English

- Extract hypothesis

In [38]:
!grep ^H en-de.decode_Beam_10.log | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp.txt

In [39]:
!head ./hyp.txt

Er fuhr das Auto
Er fuhr mit dem Auto
Er steuerte das Auto
Er fuhr den Wagen
Er lenkte das Auto
Er fuhr Auto
Er steuerte den Wagen
Er fuhr das Auto .
Er hat das Auto gefahren
Er führte das Auto


In [40]:
# Dividing tokenized text into subword units

!$FASTBPE applybpe bpe.hyp.de hyp.txt bpecodes.de

print('Finished subword.')

Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from hyp.txt ...
Read 42 words (14 unique) from text file.
Applying BPE to hyp.txt ...
Modified 42 words from text file.
Finished subword.


In [41]:
# !!! Copy the dict.en to data-bin for translation
!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_de-en \
    --workers 8

2023-03-20 17:27:24 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin_de-en', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='de', srcdict='/exp

- Translate hypothesis back to En

In [31]:
MODELS="/export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble"
NBEST = 10
BEAM = 10

# Generate N hypothesis
!fairseq-generate data-bin_de-en  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 256 \
    --memory-efficient-fp16 \
    --print-alignment \
    --remove-bpe > de-en.decode_Beam_10_backtranslation.log

2023-03-23 21:57:43 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

# Statistics

In [13]:
PATH="/export/data4/vzhekova/biases-data/Test_De/Ambiguity"

In [14]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De/Ambiguity


- Extract hypothesis

In [15]:
!grep ^H en-de.decode_Beam_10.log | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp.txt

In [16]:
# Original sentence: The doctor asked the patient.
!head ./hyp.txt

Er fuhr das Auto
Er fuhr mit dem Auto
Er steuerte das Auto
Er fuhr den Wagen
Er lenkte das Auto
Er fuhr Auto
Er steuerte den Wagen
Er fuhr das Auto .
Er hat das Auto gefahren
Er führte das Auto


In [17]:
!grep ^H de-en.decode_Beam_10_backtranslation.log | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_back.txt

In [18]:
!cat ./hyp_back.txt

He drove the car
He was driving the car
He drove by car
He was driving a car
He was driving his car
He drove his car
He drove with the car
He drove a car
He went by car
He was driving
He was driving the car
He steered the car
He controlled the car
He was steering the car
He was controlling the car
He was driving the car .
He was piloting the car
He piloted the car
He drove the car
He controlled the car .
He steered the car
He was driving the car
He was steering the car
He was driving the car .
He drove the car
He drove the car .
He was controlling the car
It steered the car
He directed the car
He controlled the car
He steered the car
He was driving the car
He was steering the car
He was driving the car .
He piloted the car
He drove the car
He controlled the car
He was controlling the car
He drove the car .
He controlled the car .
He drove the car
He was driving the car
He drove the car .
He rode the car
He was driving that car
He drove that car
he drove the car
He drove his car
He driv

In [25]:
!awk '{print $(NF)}' hyp_back.txt > words_car.txt

In [26]:
text_file = open('words_car.txt', 'r')
text = text_file.read()

def word_count(str):
    counts = dict()
    words = str.split()

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts

print( word_count(text))

{'car': 62, 'driving': 2, '.': 25, 'it': 2, 'cart': 1, 'wagon': 1, 'carriage': 1, 'vehicle': 1, 'van': 1, 'truck': 1, 'cars': 2, 'drove': 1}
